In [1]:
from dotenv import load_dotenv 
from azure.identity import DeviceCodeCredential
from msgraph import GraphServiceClient
from msgraph.generated.users.item.messages.messages_request_builder import MessagesRequestBuilder
from msgraph.generated.search.query.query_post_request_body import QueryPostRequestBody
from msgraph.generated.search.query.query_request_builder import QueryRequestBuilder
from msgraph.generated.search.search_request_builder import SearchRequestBuilder
from msgraph.generated.models.search_request import SearchRequest
from msgraph.generated.models.entity_type import EntityType
from msgraph.generated.models.search_query import SearchQuery
from msgraph.generated.models.search_response import SearchResponse
import os
import json
import asyncio 
import uuid

In [2]:
from typing import Tuple

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding

In [3]:
kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI service used by the kernel
api_key, org_id = sk.openai_settings_from_dot_env()
#kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))
kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-4", api_key, org_id))
kernel.add_text_embedding_generation_service("ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())

{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x115217d10>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x115217110>}

In [4]:
#print(org_id)

In [5]:
load_dotenv()

True

In [6]:
COLLECTION = "ESPC23"

In [7]:
TENANT_ID = os.getenv("TENANT_ID")

In [8]:
CLIENT_ID = os.getenv("CLIENT_ID")

In [9]:
graphUserScopes = os.getenv("graphUserScopes").split(' ')

In [10]:
#print(CLIENT_ID)

In [11]:
gabby = '9c72876c-b0ec-41b3-8079-8ebe216daa34'

In [12]:
#fabs = 'fabian@fabster.onmicrosoft.com'

In [13]:
## ensure that you have turned on DefaultClientType to Yes in the App Registration

In [14]:
print(graphUserScopes)

['User.Read', 'Mail.ReadWrite', 'Chat.ReadWrite', 'Calendars.Read', 'Files.Read.All', 'Sites.Read.All']


In [15]:
credential = DeviceCodeCredential(client_id=CLIENT_ID, tenant_id=TENANT_ID) 
scopes = graphUserScopes
client = GraphServiceClient(credentials=credential, scopes=scopes)

In [17]:
result = await client.me.get()

In [18]:
print(result.display_name)

Gabby Gabgab


## Do the Query and Get the Intent of the User Asking the Question (UserPrompt)

In [19]:
# note: Using a File to Run these Plugins instead of Inline to 
# show a different way
plugins_directory = "Plugins/"
intentFunctions = kernel.import_semantic_skill_from_directory(plugins_directory, "HoldMyHandPlugin")

getIntentFunction = intentFunctions["GetIntent"]

In [20]:
userIntent = getIntentFunction("What did I accomplish this weekend with Gabby in Antwerp regarding our ESPC demo?")

print(userIntent)

{ "subject": "Gabby", "action": "know what was accomplished during the ESPC demo in Antwerp"}


In [21]:
subject_data = json.loads(str(userIntent))
print(subject_data)

{'subject': 'Gabby', 'action': 'know what was accomplished during the ESPC demo in Antwerp'}


In [22]:
subject = subject_data['subject']
action = subject_data['action']

In [23]:
print(subject)

Gabby


In [24]:
print(action)

know what was accomplished during the ESPC demo in Antwerp


## Work with Teams(Chat) Search Query Endpoint

In [25]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.ChatMessage
			],
			query = SearchQuery(
				query_string = subject,
			),
		),
	],
)

chat_result = await client.search.query.post(request_body)

In [26]:
chat_data = chat_result.value

In [27]:
##Testing with GUID
search_response = chat_data[0] 

# Initialize an empty list to store summaries, chat IDs, and generated GUIDs
summaries_chat_ids_and_guids = []

# Iterate over hits_containers in the SearchResponse object
for hits_container in search_response.hits_containers:
    # Iterate over each hit in the container
    for hit in hits_container.hits:
        # Extract the summary
        summary = hit.summary

        # Extract the chatId from additional_data
        chat_id = hit.resource.additional_data.get('chatId', 'Unknown')

        # Generate a unique GUID for each hit
        guid = str(uuid.uuid4())

        # Append the chatId, summary, and GUID as a tuple to the list
        summaries_chat_ids_and_guids.append((guid, summary))

# Now 'summaries_chat_ids_and_guids' contains the chatId, summary, and a unique GUID for each search hit
for item in summaries_chat_ids_and_guids:
    await kernel.memory.save_information_async("ESPC23", id=guid, text=summary)
    #print(item)

                                            

In [ ]:
#print(summaries_chat_ids_and_guids)

## Work with Drives(Files) Search Query Endpoint

In [28]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.DriveItem
			],
			query = SearchQuery(
				query_string = subject,
			),
		),
	],
)

drive_result = await client.search.query.post(request_body)

In [29]:
drive_data = drive_result.value
#print(drive_data)

In [30]:
try:
    drive_search_response = drive_data[0]  # Assuming there's only one SearchResponse object in the list

    # Initialize an empty list to store summaries, from information, and generated GUIDs
    drive_summary_from_and_guids = []
    drive_narrative = []

    if drive_search_response:
        # Iterate over hits_containers in the SearchResponse object
        for drive_hits_container in drive_search_response.hits_containers:
            # Iterate over each hit in the container
            for drive_hit in drive_hits_container.hits:
                # Extract the summary
                drive_summary = drive_hit.summary

                # Extract the 'from' information from additional_data
                from_data = drive_hit.resource.additional_data.get('from', {}).get('emailAddress', {})
                from_name = from_data.get('name', 'Unknown')
                from_email = from_data.get('address', 'Unknown')
                file_name = getattr(drive_hit.resource, 'name', 'Unknown')
                user_email = from_data.get('email', 'Unknown')

                # Generate a unique GUID for each hit
                drive_guid = str(uuid.uuid4())

                # Append the summary, 'from' information, and GUID as a tuple to the list
                drive_summary_from_and_guids.append((drive_guid, drive_summary, from_name, from_email))
                drive_narrative.append((drive_guid, file_name))
except Exception as e:
    print(f"An error occurred: {e}")

# Now 'drive_summary_from_and_guids' contains the summary, from information, and a unique GUID for each search hit
for item in drive_narrative:
    await kernel.memory.save_information_async("ESPC23", id=drive_guid, text=file_name)
    #print(item)



In [31]:
#print(drive_narrative)

## Work with Messages(Email) Search Query Endpoint

In [32]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.Message
			],
			query = SearchQuery(
				query_string = subject,
			),
		),
	],
)

mail_result = await client.search.query.post(request_body)

In [33]:
mail_data = mail_result.value

In [34]:
try:
    mail_search_response = mail_data[0]  # Assuming there's only one SearchResponse object in the list

    # Initialize an empty list to store summaries, from information, and generated GUIDs
    mail_summary_from_and_guids = []

    if mail_search_response:
        # Iterate over hits_containers in the SearchResponse object
        for mail_hits_container in mail_search_response.hits_containers:
            # Iterate over each hit in the container
            for mail_hit in mail_hits_container.hits:
                # Extract the summary
                mail_summary = mail_hit.summary

                # Extract the 'from' information from additional_data
                from_data = mail_hit.resource.additional_data.get('from', {}).get('emailAddress', {})
                from_name = from_data.get('name', 'Unknown')
                from_email = from_data.get('address', 'Unknown')

                # Generate a unique GUID for each hit
                guid = str(uuid.uuid4())

                # Append the summary, 'from' information, and GUID as a tuple to the list
                mail_summary_from_and_guids.append((guid, mail_summary, from_name, from_email))
                mailconstruct = mail_summary + " from " + from_name
except Exception as e:
    print(f"An error occurred: {e}")

# Now 'mail_summary_from_and_guids' contains the summary, from information, and a unique GUID for each search hit
for item in mail_summary_from_and_guids:
    await kernel.memory.save_information_async("ESPC23", id=guid, text=mailconstruct)
    #print(item)

In [35]:
#print(mail_summary_from_and_guids)

## Query and Read All 3 Payloads from Semantic Memory ONLY

In [36]:
async def search_memory_examples(kernel: sk.Kernel) -> None:
    questions = [
        "Whats going on with Gabby",
        "What conferences are we doing",
    ]

    for question in questions:
        results = await kernel.memory.search_async("ESPC23", question, limit=3)
        for result in results:
            print(f"Question: {question}")
            print(f"Answer: {result.text}\nRelevance: {result.relevance}\n")

In [48]:
await search_memory_examples(kernel)

Question: Whats going on with Gabby
Answer: hello Gabby, are you prepared for the ChatGPT session with Python?
Relevance: 0.8015434058556261

Question: Whats going on with Gabby
Answer: Ok, So make sure you know your lines, study your answers to the questions. Get your demo ready to go. MOST IMPORTANT: do all the sacrifices to the demo gods!!! Dad. from Unknown
Relevance: 0.7223546610199888

Question: Whats going on with Gabby
Answer: HowToGainAccessToStudyGroup.docx
Relevance: 0.7196357848102334

Question: What conferences are we doing
Answer: HowToGainAccessToStudyGroup.docx
Relevance: 0.7515025470786859

Question: What conferences are we doing
Answer: hello Gabby, are you prepared for the ChatGPT session with Python?
Relevance: 0.7329083577823825

Question: What conferences are we doing
Answer: Ok, So make sure you know your lines, study your answers to the questions. Get your demo ready to go. MOST IMPORTANT: do all the sacrifices to the demo gods!!! Dad. from Unknown
Relevance: 0.

## Setup RAG Function

In [49]:
print(COLLECTION)

ESPC23


In [54]:
async def setup_RAG(kernel: sk.Kernel) -> Tuple[sk.SKFunctionBase, sk.SKContext]:
    sk_prompt = """
    You are a friendly and very talkative ChatBot. 
    
    {{$history}}
    Answer the {{$user_input}}. You may use information provided in {{$retreived_context}} it may be useful. 
    Return complete answers wiht prices only from {{$retreived_context}} otherwise say 'I have no reccomendations for you'. 
    ChatBot:""".strip()

    rag_func = kernel.create_semantic_function(sk_prompt, max_tokens=2000, temperature=0.1)

    context = kernel.create_new_context()

    return rag_func, context

In [55]:
async def RAG(kernel: sk.Kernel, rag_func: sk.SKFunctionBase, context: sk.SKContext) -> bool:

    user_input = input("User:> ")
    context["user_input"] = user_input
    #Retrieve
    result = await kernel.memory.search_async(COLLECTION,context["user_input"])

    
    if result[0].text:
        context["retreived_context"] = result[0].text
    
    #Then generate
    answer = await kernel.run_async(rag_func, input_vars=context.variables)

    print(f"\n\u001b[34mChatBot:> {answer}\u001b[0m \n\n\033[1;32m Source: {context['retreived_context']}\n Relevance: {result[0].relevance}\u001b[0m \n")
    return False

In [56]:
print("Simulate Chat experience with SK and RAG")
rag_func, context = await setup_RAG(kernel)

Simulate Chat experience with SK and RAG


In [60]:
print("Ask a question (type 'exit' to exit):\n")
answer = await RAG(kernel, rag_func, context)

Ask a question (type 'exit' to exit):



User:>  What did fabian and gabby work on 



ChatBot:> Based on the information provided, Fabian and Gabby worked on a ChatGPT session with Python. 

 Source: hello Gabby, are you prepared for the ChatGPT session with Python?
 Relevance: 0.7917456309860029 

